# Template - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
# from backtesting._plotting import set_bokeh_output
# set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [ ]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume,ohcl,ohcl_p
Date,,,,,,,
2018-08-13 09:00:00,943.5,943.6,942.9,943.1,1812,943.275,943.200000
2018-08-13 09:05:00,943.1,943.5,942.9,943.3,1323,943.200,943.233333
2018-08-13 09:10:00,943.2,943.3,942.6,943.1,1207,943.050,943.000000
2018-08-13 09:15:00,943.1,943.1,942.3,942.6,1196,942.775,942.666667
2018-08-13 09:20:00,942.6,943.7,942.4,943.7,1765,943.100,943.266667
...,...,...,...,...,...,...,...
2024-07-16 14:15:00,1300.5,1302.1,1299.1,1302.0,9624,1300.925,1301.066667
2024-07-16 14:20:00,1302.0,1303.4,1301.1,1302.4,7682,1302.225,1302.300000
2024-07-16 14:25:00,1302.4,1302.4,1301.1,1301.8,4741,1301.925,1301.766667


In [ ]:
def cal_signal(r):
    signal = ''
    if r['Close'] > r['Close_s1']:
        signal = 'long'
    elif r['Close'] > r['Close_s1']:
        signal = 'long'
    return signal

def prepare_data(data):
    data['Close_s1'] = data['Close'].shift(1)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    return data

In [ ]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [ ]:
class MainStrategy(Strategy):
    reward_on_risk = 3
    def init(self):
        self._broker._cash = 1500
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if self.position:
            return 
        signal = self.data.signal[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - 3
            tp = buy_price + 6
            self.buy(size=1, sl=sl, tp=tp)
            # self.buy(size=1, sl=sl, tp=tp, limit=buy_price)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + 3
            tp = sell_price - 6
            self.sell(size=1, sl=sl, tp=tp)
            # self.sell(size=1, sl=sl, tp=tp, limit=sell_price)

In [ ]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [ ]:
stats

In [ ]:
stats['_trades']

In [ ]:
# bt.plot()

In [ ]:
copy_trades = stats['_trades'].copy()
copy_trades['cum_sum'] = copy_trades['PnL'].cumsum()
X = np.array(range(0, len(copy_trades['cum_sum'])))
Y = copy_trades['cum_sum']
# Plotting the Graph
plt.plot(X, Y)
plt.title("Curve plotted for returns")
plt.xlabel("X")
plt.ylabel("Rerurns")
plt.show()